#### SQLite port_lite database: sales, stocks table
#### csv files: daily-watch-list, daily-sales, monitors, stocks-all

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
yesterday

datetime.date(2022, 2, 22)

### Yesterday = Last closed business day

In [2]:
#yesterday = date(2022, 2, 11)
yesterday

datetime.date(2022, 2, 22)

### Restart and Run All Cells

In [3]:
cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

In [4]:
format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [5]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1108279,SNC,2022-02-21,2022-02-22,1,17.80,17.00,-0.80,-4.49%,-8,-8,17.70,16.80,"2,889,075",2022-02-22,2022-02-22,1


In [6]:
names = tmp["name"]
name = names.to_string(index=False)
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'SNC'



TypeError: unsupported format string passed to NoneType.__format__

### End of Tables in the process

In [7]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date >= '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date >= '2022-02-22' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [8]:
df = pd.read_sql(sql, conlite)
df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['diff'] = (df['to_price'] - df['fm_price'])
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.head().style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
name,,,,,,,,,,,
MEGA,2022-02-18,2022-02-22,43.50,42.75,"6,398,392",43.25,-0.57,B,SET100,-0.75,-1.72%
NOBLE,2022-02-18,2022-02-22,6.25,6.20,"3,971,959",6.25,-0.80,B,SET,-0.05,-0.80%
PTTGC,2022-02-09,2022-02-22,58.25,54.75,"110,891,926",55.00,-5.58,B,SET50,-3.50,-6.01%
SINGER,2022-02-18,2022-02-22,42.25,44.50,"8,373,772",44.75,2.37,B,SET100,2.25,5.33%
TMT,2022-02-09,2022-02-22,10.20,10.80,"7,321,601",10.90,5.88,B,SET,0.60,5.88%


### Price change up or down more than 3%

In [9]:
#df['bdays'] = np.busday_count(df['fm_date'],df['to_date'])
#df['bdays'] = 1
df['bdays'] = (df['to_date'] - df['fm_date']).dt.days
watch_list = df.loc[
    abs(df.percent) >= 3,
    ['percent','status','fm_price','to_price','market','fm_date','bdays','max_price','min_price']
]
watch_list.sort_values(by="percent",ascending=False).style.format(format_dict)

,percent,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
name,,,,,,,,,
RCL,7.88%,I,41.25,44.50,SET100,2022-02-14,8,44.75,6.06
LPF,6.15%,I,13.00,13.80,SET,2022-02-14,8,13.80,4.62
TMT,5.88%,B,10.20,10.80,SET,2022-02-09,13,10.90,5.88
SINGER,5.33%,B,42.25,44.50,SET100,2022-02-18,4,44.75,2.37
NER,4.90%,I,7.15,7.50,SET,2022-02-14,8,7.65,4.20
TSE,4.84%,I,2.48,2.60,SET,2022-02-11,11,2.84,1.61
MAKRO,3.59%,O,41.75,43.25,SET,2022-02-17,5,43.50,3.59
ASK,3.37%,O,44.50,46.00,SET,2022-02-18,4,46.00,1.12
SPALI,-4.27%,O,23.40,22.40,SET100,2022-02-15,7,22.60,-4.27


In [10]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

watch_list.sort_values(['percent'],ascending=[False]).to_csv(output_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(data_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(box_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(one_file)

### Create daily-sales from sales

In [11]:
df[cols].head().style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
name,,,,,,,,,
MEGA,2022-02-18,2022-02-22,43.50,42.75,"6,398,392",43.25,-0.57,-1.72%,B
NOBLE,2022-02-18,2022-02-22,6.25,6.20,"3,971,959",6.25,-0.80,-0.80%,B
PTTGC,2022-02-09,2022-02-22,58.25,54.75,"110,891,926",55.00,-5.58,-6.01%,B
SINGER,2022-02-18,2022-02-22,42.25,44.50,"8,373,772",44.75,2.37,5.33%,B
TMT,2022-02-09,2022-02-22,10.20,10.80,"7,321,601",10.90,5.88,5.88%,B


In [12]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(box_file)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file)

### Create monitors from stocks

In [13]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.head()

,status
name,
ASK,O
ASP,O
DOHOME,I
GLOBAL,I
IP,I


In [14]:
sql = """
SELECT status, COUNT(*) AS 'no. of stocks'
FROM stocks 
WHERE status IN ("B","I","O","S")
GROUP BY status
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,status,no. of stocks
0,B,5
1,I,18
2,O,10
3,S,1


### output_file = iCloud Drive on PC

In [15]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)
monitors.to_csv(output_file)
monitors.to_csv(one_file)

### End of daily process

In [16]:
sql = """
SELECT status, COUNT(*) AS 'no. of stocks'
FROM stocks 
GROUP BY status
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,status,no. of stocks
0,B,5
1,I,18
2,O,10
3,S,1
4,T,5
5,U,3
6,X,14


In [17]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
MEGA,B,SET100
NOBLE,B,SET
PTTGC,B,SET50
SINGER,B,SET100
TMT,B,SET
DOHOME,I,SET100
GLOBAL,I,SET50
IP,I,mai
IVL,I,SET50


In [18]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [19]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
ASK,O
ASP,O
KKP,O
MAKRO,O
MEGA,B
NOBLE,B
PTTGC,B
ROJNA,O
SCCC,O


In [20]:
buy_candidates.shape[0]

15

In [21]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
DOHOME,I
GLOBAL,I
IP,I
IVL,I
LPF,I
MCS,I
NER,I
PTT,I
RATCH,I


In [22]:
sell_candidates.shape[0]

19